In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import numpy as np
from source.utils.load_utils import just_load
from source.datasets.pretrain_datasets import MSDDatasetPretrain
from source.models_pretraining.mb_mlm_split import MusicBertMLM_split

# REVIEWS DATASET
song_dataset = MSDDatasetPretrain()

train_dataloader = just_load(song_dataset, workers=8, batch_size=4)

evals = 1
model = MusicBertMLM_split(CLS_training='MOP', 
                            MLM_training='MSE',
                            num_encoder_layers=4,
                            name="music_bert_mlm"
                           ).cuda()
#model.load_pretrained()

In [2]:
model.pretrain_model(train_dataloader, None, epochs=1, eval_per_epoch=evals)

Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 234, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "/usr/lib/python3.6/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
  File "/usr/local/lib/python3.6/dist-packages/torch/multiprocessing/reductions.py", line 333, in reduce_storage
    fd, size = storage._share_fd_()
RuntimeError: unable to write to file </torch_65319_1978034267>
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 234, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "/usr/lib/python3.6/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
  File "/usr/local/lib/python3.6/dist-packages/torch/multiprocessing/reductions.py", line 333, in reduce_storage
    fd, size = storage._share_fd_()
RuntimeError: unable to write to file </torch_65320_448159986>
Traceback (most recent call last):
  File "/usr/lib/python3.6/mul

RuntimeError: DataLoader worker (pid(s) 65322) exited unexpectedly

In [ ]:
from source.utils.generic_utils import allDone

allDone()

In [15]:
%matplotlib widget

from source.utils.plot_utils import plot_curve, smooth

cls_loss = smooth(model.cls_loss_curve.cpu().numpy(), 100)
mlm_loss = smooth(model.mask_loss_curve.cpu().numpy(), 10)

plot_curve(cls_loss, 1, color="red")
plot_curve(mlm_loss/15, 1, color="blue")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
%load_ext line_profiler

%lprun -f MSDDatasetPretrain.__getitem__ model.pretrain_model(train_dataloader, None, epochs=1, eval_per_epoch=evals)

*** KeyboardInterrupt exception caught in code being profiled.

Timer unit: 1e-06 s

Total time: 33.0413 s
File: /nfs/notebooks/source/dataset_pretrain.py
Function: __getitem__ at line 42

Line #      Hits         Time  Per Hit   % Time  Line Contents
    42                                               def __getitem__(self, idx):
    43                                                   
    44      1843       4761.0      2.6      0.0          msd_id = self.msd_ids_list[idx][0:17] #Cutoff to comply with moodagent modified IDs
    45      1843       1842.0      1.0      0.0          msd_id = msd_id.encode()
    46                                                   
    47      1843       7916.0      4.3      0.0          with self.song_env.begin(write=False) as txn:
    48      1843   32982999.0  17896.4     99.8              data = txn.get(msd_id)
    49                                                       
    50      1842       2771.0      1.5      0.0          assert data is not None, "Key Error in database"
    51                               